In [1]:
import numpy as np
import pandas as pd
from msresist.clustering import MassSpecClustering
from msresist.pre_processing import filter_NaNpeptides
from msresist.figures.figureM4 import find_patients_with_NATandTumor
from sklearn.preprocessing import StandardScaler

X = pd.read_csv("msresist/data/MS/CPTAC/CPTAC-preprocessedMotfis.csv").iloc[:, 1:]
X = filter_NaNpeptides(X, cut=1)
d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

for _ in range(10):
    try:
        model_min = MassSpecClustering(i, ncl=15, SeqWeight=2, distance_method="Binomial").fit(d, "NA")
    except:
        continue


Matplotlib is building the font cache; this may take a moment.


In [2]:
pd.DataFrame(model_min.transform()).T

NameError: name 'model_min' is not defined